In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

Tensorflow version 2.8.2
Running on TPU  ['10.107.7.90:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.107.7.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.107.7.90:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [4]:
tf.__version__

'2.8.2'

In [37]:
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
import os

In [38]:
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [8]:
train = pd.read_csv('train.csv')

# New Section

In [9]:
test = pd.read_csv('test.csv')

In [10]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [11]:
import re

In [12]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts

In [13]:
train.text = normalize_texts(train.text)

In [14]:
test.text = normalize_texts(test.text)

In [21]:
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'


In [28]:
preprocessor_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [39]:
bert_preprocess_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [40]:
bert_model = hub.KerasLayer(encoder_url, trainable=False)

In [81]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

layer = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
layer2 = tf.keras.layers.Dense(32, activation='relu')(layer)
layer3 = tf.keras.layers.Dropout(0.4, name="dropout2")(layer2)



final_layer = tf.keras.layers.Dense(1, activation='sigmoid', name="final")(layer3)

model = tf.keras.Model(inputs=[text_input], outputs=[final_layer])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [82]:

metrics =tf.metrics.BinaryAccuracy()

In [83]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=metrics)

In [84]:
train_text = train.text.values

In [85]:
test_text = test.text.values

In [86]:
train_label = train.target.values

In [87]:
history = model.fit(train_text,train_label, validation_split=0.2, epochs=15)

Epoch 1/15
191/191 [==============================] - 224s 1s/step - loss: 0.6488 - binary_accuracy: 0.6144 - val_loss: 0.6155 - val_binary_accuracy: 0.6704
Epoch 2/15
191/191 [==============================] - 218s 1s/step - loss: 0.6107 - binary_accuracy: 0.6670 - val_loss: 0.6094 - val_binary_accuracy: 0.6724
Epoch 3/15
191/191 [==============================] - 219s 1s/step - loss: 0.5965 - binary_accuracy: 0.6849 - val_loss: 0.5583 - val_binary_accuracy: 0.7610
Epoch 4/15
191/191 [==============================] - 217s 1s/step - loss: 0.5831 - binary_accuracy: 0.7082 - val_loss: 0.5434 - val_binary_accuracy: 0.7708
Epoch 5/15
191/191 [==============================] - 217s 1s/step - loss: 0.5820 - binary_accuracy: 0.7061 - val_loss: 0.5545 - val_binary_accuracy: 0.7518
Epoch 6/15
191/191 [==============================] - 217s 1s/step - loss: 0.5728 - binary_accuracy: 0.7182 - val_loss: 0.5292 - val_binary_accuracy: 0.7400
Epoch 7/15
191/191 [==============================] - 219s

In [88]:
submission = pd.read_csv('sample_submission.csv')

In [89]:
test_pred = model.predict(test_text)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)